In [18]:
import numpy as np

In [19]:
## Helper Functions ##
def sigmoid(x):
    return 1/(1 + np.exp(-x))

def softmax(x):
    return np.exp(x) / np.sum(np.exp(x)) 

def relu(x):
    return x * (x > 0)

def deriv_sigmoid(x):
    fx = sigmoid(x)
    return fx * (1 - fx)

def deriv_loss(y_true, y_pred):
    return -2*(y_true - y_pred)

def deriv_pred(hidden, o_weight, o_bias):
    return o_weight * deriv_sigmoid(np.dot(o_weight, hidden) + o_bias)[:,None]

def deriv_hidden(i_data, i_weight, i_bias):
    n_dim = i_weight.shape[0]
    w_dim = i_weight.shape[1]
    derv = np.zeros((n_dim, n_dim * w_dim + n_dim))
    for i in range(0,n_dim):
        derv[i : i+1, i*w_dim : (i+1)*w_dim] = np.array(i_data) * deriv_sigmoid(np.dot(i_weight[i,:],i_data) + i_bias[i])
        derv[i : i+1, n_dim*w_dim+i : n_dim*w_dim+i+1] = deriv_sigmoid(np.dot(i_weight[i,:],i_data) + i_bias[i])
    return derv

def mse_loss(y_true, y_pred):
    return ((y_true - y_pred) ** 2).mean()

def gradient_descent(i_weight, o_weight, i_bias, o_bias, h_field, o_field, h_bias_field, o_bias_field, learning_rate):
    i_weight = i_weight - learning_rate * h_field
    o_weight = o_weight - learning_rate * o_field
    i_bias = i_bias - learning_rate * h_bias_field
    o_bias = o_bias - learning_rate * o_bias_field
    return i_weight, o_weight, i_bias, o_bias

In [20]:
## Neuron Class Object ##
class Neuron:
    def __init__(self, weight, bias):
        self.weight = weight
        self.bias = bias
        
    def feedforward(self, inputs, act_type):
        total = np.dot(self.weight, inputs) + self.bias
        if act_type == 0:
            return sigmoid(total)
        elif act_type == 1:
            return softmax(total)
        elif act_type == 2:
            return relu(total)

In [21]:
## Neural Network Class Object ##
class Neural_Net:
    def __init__(self, h_num, o_num, i_weight, o_weight, i_bias, o_bias):
        self.i_weight = i_weight
        self.o_weight = o_weight
        self.i_bias = i_bias
        self.o_bias = o_bias
        self.h_num = h_num
        self.o_num = o_num
        self.hidden = []
        self.output = []
        for i in range(h_num):
            self.hidden.append(Neuron(i_weight[i,:], i_bias[i]))
        for i in range(o_num):
            self.output.append(Neuron(o_weight[i,:], o_bias[i]))
    
    def __del__(self):
            
    def feedforward(self, i_data):
        o_hidden = []
        o_output = []
        for i in range(self.h_num):
            o_hidden.append(self.hidden[i].feedforward(i_data,0))
        for i in range(self.o_num):
            o_output.append(self.output[i].feedforward(o_hidden,0))
        return o_output, o_hidden
    
    def backpropagation(self, i_y, y_pred, h_value, i_x):
        h_field = np.matmul(np.matmul(deriv_loss(i_y,y_pred), deriv_pred(h_value,self.o_weight,self.o_bias)), deriv_hidden(i_x, self.i_weight, self.i_bias))
        o_field = np.matmul(deriv_loss(i_y,y_pred), deriv_hidden(h_value, self.o_weight, self.o_bias))
        h_bias_field = h_field[len(h_field)-self.h_num:]
        h_field = np.reshape(h_field[0:len(h_field)-self.h_num], (self.h_num, len(i_x)))#h_field[0:len(h_field)-self.h_num].reshape[self.h_num, self.o_num]
        o_bias_field = o_field[len(o_field)-self.o_num:]
        o_field = np.reshape(o_field[0:len(o_field)-self.o_num], (self.o_num, self.h_num))
        return h_field, o_field, h_bias_field, o_bias_field

In [62]:
## Training Function ##
def train(i_weight, o_weight, i_bias, o_bias, i_data, learning_rate, epoch):
    h_size = i_weight.shape[0]
    o_size = i_data.shape[1]
    n = i_data.shape[1]
    obs_n = i_data.shape[0]
    np.random.shuffle(i_data)
    i_y = i_data[: , n-1:n]
    o_size = i_y.shape[1]
    i_x = i_data[: , 0:n-1]
    
    for j in range(epoch):
        loss = 0
        for i in range(0,obs_n):
            network = Neural_Net(h_size, o_size, i_weight, o_weight, i_bias, o_bias)
            pred_y, h_value = network.feedforward(i_x[i, :])
            h_field, o_field, h_bias_field, o_bias_field = network.backpropagation(i_y[i], pred_y, h_value, i_x[i, :])
            i_weight, o_weight, i_bias, o_bias = gradient_descent(i_weight, o_weight, i_bias, o_bias, h_field, o_field, h_bias_field, o_bias_field, learning_rate)
            loss = loss + mse_loss(i_y[i], pred_y)
            if (j != epoch - 1 or i != obs_n - 1):
                del network
        print("Epoch %d loss: %.3f" % (j + 1, loss))
    return network

In [70]:
## Network Parameter Initialization ##
#i_weight = np.array([[0.3,0.7],[0.1,0.2],[0.4,0.1],[0.2,0.5],[0.3,0.7],[0.1,0.2],[0.4,0.1],[0.2,0.5],[0.4,0.1],[0.2,0.5]])
#o_weight = np.array([[0.4,0.3,0.2,0.1,0.4,0.3,0.2,0.1,0.4,0.5]])
#i_bias = np.array([0.5,0.4,0.3,0.5,0.4,0.3,0.5,0.4,0.3,0.5])
#o_bias = np.array([1])
x_i_weight = 100
y_i_weight = 2
i_weight = np.random.normal(size=(x_i_weight,y_i_weight))

x_o_weight = 1
y_o_weight = 100
o_weight = np.random.normal(size=(x_o_weight,y_o_weight))

x_i_bias = 1
y_i_bias = 100
i_bias = np.random.normal(size=(y_i_bias))

x_o_bias = 1
y_o_bias = 1
o_bias = np.random.normal(size=(x_o_bias))

## Input Data ##
import csv
with open('WeightHeight.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',',quoting=csv.QUOTE_NONNUMERIC)
    sex = []
    weight = []
    height = []
    for row in readCSV:
        sex.append(row[0])
        height.append(row[1])
        weight.append(row[2])
        
sex = np.array(sex[1:])
height = np.subtract(height[1:],66)
weight = np.subtract(weight[1:],135)
sex = np.where(sex=='Male',0,1)
i_data = np.transpose(np.array([weight, height, sex]))
np.random.shuffle(i_data)
print(i_data)

[[ 27.40100084   0.88975886   1.        ]
 [ 17.89696485   1.87840172   0.        ]
 [ 48.4010746    2.74134888   0.        ]
 ...
 [ 57.17213679   3.1330814    0.        ]
 [-20.03096175  -3.14962136   1.        ]
 [ 81.14921773   7.22307094   0.        ]]


In [71]:
## Training ##
network=train(i_weight, o_weight, i_bias, o_bias, i_data, 0.1, 10)

Epoch 1 loss: 1476.864
Epoch 2 loss: 1562.773
Epoch 3 loss: 1460.574
Epoch 4 loss: 1535.263
Epoch 5 loss: 1506.128
Epoch 6 loss: 1497.858
Epoch 7 loss: 1448.336
Epoch 8 loss: 1382.300
Epoch 9 loss: 1423.952
Epoch 10 loss: 1437.806


In [72]:
## Prediction ##
Boy = np.array([45, 8]) # 180 pounds, 74 inches
Girl = np.array([6, 5])  # 141 pounds, 71 inches
p1,h1 = network.feedforward(Boy)
p2,h2 = network.feedforward(Girl)
print("Boy: %.3f" % p1[0])
print("Girl: %.3f" % p2[0])

Boy: 0.002
Girl: 0.000


In [73]:
## Need some work still ;) ##